# Mystery #1 - The Phantom Profits

## As the end of the 2016 fiscal year approached, the Accounting Team had realized that the records were not adding up, high-end transactions were disappearing and orders were being modified. It had become apparent that someone on the inside was siphoning funds. It's your job to find out who this thief within the company is.

Task 1 (Query 1): Determine the biggest purchase made in 2016

\- Retrieves the top 10 transactions, displaying transaction details, customer information, transaction type, and employee  <span style="color: var(--vscode-foreground);">processor</span>

\- Filters Year to 2016 to sort through highest to lowest transactions

\- Accomplished through joins of Sales.CustomerTransactions, <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Sales.Customers, Application.TransactionTypes, Application.People</span>

In [97]:
SELECT TOP 10
    ct.CustomerTransactionID,
    ct.CustomerID,
    ct.TransactionAmount,
    ct.TransactionDate,
    c.CustomerName,
    tt.TransactionTypeName,
    p.FullName AS ProcessedBy
FROM Sales.CustomerTransactions ct
INNER JOIN Sales.Customers c ON ct.CustomerID = c.CustomerID
INNER JOIN Application.TransactionTypes tt ON ct.TransactionTypeID = tt.TransactionTypeID
INNER JOIN Application.People p ON ct.LastEditedBy = p.PersonID
WHERE YEAR(ct.TransactionDate) = 2016
ORDER BY ct.TransactionAmount DESC

(10 rows affected)

Total execution time: 00:00:00.064

CustomerTransactionID,CustomerID,TransactionAmount,TransactionDate,CustomerName,TransactionTypeName,ProcessedBy
306399,1,33966.40,2016-02-20,Tailspin Toys (Head Office),Customer Invoice,Jack Potter
319701,1043,31489.30,2016-04-07,Raj Verma,Customer Invoice,Kayla Woodcock
329234,401,31470.90,2016-05-07,Wingtip Toys (Head Office),Customer Invoice,Eva Muirden
335968,401,28280.80,2016-05-31,Wingtip Toys (Head Office),Customer Invoice,Hudson Onslow
314402,1,27219.12,2016-03-21,Tailspin Toys (Head Office),Customer Invoice,Sophia Hinton
294972,1056,25334.94,2016-01-07,Kalyani Benjaree,Customer Invoice,Lily Code
296180,1,25247.55,2016-01-12,Tailspin Toys (Head Office),Customer Invoice,Anthony Grosse
330404,401,24735.80,2016-05-12,Wingtip Toys (Head Office),Customer Invoice,Hudson Onslow
302334,983,24110.90,2016-02-03,Sabine Alksne,Customer Invoice,Stella Rosenhain
328915,904,23877.45,2016-05-06,Olafs Rozitis,Customer Invoice,Kayla Woodcock


Task 2 (Query 2) : Check every detail regarding the customer from Task 1  
\- Displays information about customer like identity, finances, classification, contact info, and location info  
\- Combines data through joins on Sales.Customers, Sales.CustomerCategories, Sales.BuyingGroups, Application.People, Application.Cities, Application.StateProvinces  
\- Specifically narrows down on the Customer from Task 1 via their CustomerID

In [99]:
SELECT 
    c.CustomerID,
    c.CustomerName,
    c.CreditLimit,
    cat.CustomerCategoryName,
    bg.BuyingGroupName,
    p.FullName AS PrimaryContact,
    c.PhoneNumber,
    city.CityName,
    sp.StateProvinceName
FROM Sales.Customers c
INNER JOIN Sales.CustomerCategories cat ON c.CustomerCategoryID = cat.CustomerCategoryID
LEFT JOIN Sales.BuyingGroups bg ON c.BuyingGroupID = bg.BuyingGroupID
INNER JOIN Application.People p ON c.PrimaryContactPersonID = p.PersonID
INNER JOIN Application.Cities city ON c.DeliveryCityID = city.CityID
INNER JOIN Application.StateProvinces sp ON city.StateProvinceID = sp.StateProvinceID
WHERE c.CustomerID = 1 -- Uses the CustomerID from Query 1


(1 row affected)

Total execution time: 00:00:00.019

CustomerID,CustomerName,CreditLimit,CustomerCategoryName,BuyingGroupName,PrimaryContact,PhoneNumber,CityName,StateProvinceName
1,Tailspin Toys (Head Office),NULL,Novelty Shop,Tailspin Toys,Waldemar Fisar,(308) 555-0100,Lisco,Nebraska


Task 3 (Query 3) : Go through each invoice from this Customer and look for any discrepencies

\- Shows how much was invoiced from the year 2016 via the invoice's ID, date, profit, totaled amount, and the suspected Customer ID  
\- Joines Sales.Invoices and Sales.InvoiceLines  
\- Groups all Lines belonging  to the same invoice and orders them from Highest to Lowest

In [100]:
SELECT 
    i.InvoiceID,
    i.InvoiceDate,
    SUM(il.LineProfit) AS TotalProfit,
    SUM(il.ExtendedPrice) AS TotalInvoiced,
    i.CustomerID
FROM Sales.Invoices i
INNER JOIN Sales.InvoiceLines il ON i.InvoiceID = il.InvoiceID
WHERE i.CustomerID = 1
    AND YEAR(i.InvoiceDate) = 2016
GROUP BY i.InvoiceID, i.InvoiceDate, i.CustomerID
ORDER BY TotalInvoiced DESC

(9 rows affected)

Total execution time: 00:00:00.075

InvoiceID,InvoiceDate,TotalProfit,TotalInvoiced,CustomerID
63524,2016-02-06,786.00,3174.00,1
62847,2016-01-25,1292.00,3160.20,1
70232,2016-05-27,1070.00,2530.00,1
63685,2016-02-11,792.00,1490.40,1
62922,2016-01-26,514.40,1308.58,1
62415,2016-01-18,598.00,1119.87,1
68177,2016-04-26,476.40,934.44,1
68338,2016-04-28,321.50,707.25,1
63489,2016-02-05,293.00,646.30,1


Task 4 (Query 4) : Display who's been accessing and editing these invoices the most and look for anything suspicious

\- Shows the editors for that specific CustomerID's Transactions and how often they edited it, the first and last edit, and the total sum of those transactions

\- Joins Application.People and ct.LastEditedBy

\- Groups by full name and ordered most edits

In [101]:
SELECT 
    p.FullName AS EditedBy,
    COUNT(*) AS TimesEdited,
    MIN(ct.LastEditedWhen) AS FirstEdit,
    MAX(ct.LastEditedWhen) AS LastEdit,
    SUM(ct.TransactionAmount) AS TotalAmountHandled
FROM Sales.CustomerTransactions ct
INNER JOIN Application.People p ON ct.LastEditedBy = p.PersonID
WHERE ct.CustomerID = 1
GROUP BY p.FullName
ORDER BY TimesEdited DESC

(19 rows affected)

Total execution time: 00:00:00.040

EditedBy,TimesEdited,FirstEdit,LastEdit,TotalAmountHandled
Anthony Grosse,1449,2013-01-17 11:30:00.0000000,2016-05-29 11:30:00.0000000,0.00
Stella Rosenhain,1443,2013-01-02 11:30:00.0000000,2016-05-14 11:30:00.0000000,0.00
Kayla Woodcock,1429,2013-01-09 11:30:00.0000000,2016-05-21 11:30:00.0000000,0.00
Hudson Onslow,1408,2013-01-16 11:30:00.0000000,2016-05-31 12:00:00.0000000,56435.84
Alica Fatnowna,1375,2013-01-04 11:30:00.0000000,2016-05-27 11:30:00.0000000,0.00
Jack Potter,1350,2013-01-06 11:30:00.0000000,2016-05-26 11:30:00.0000000,0.00
Henry Forlonge,1305,2013-01-13 11:30:00.0000000,2016-04-16 11:30:00.0000000,0.00
Taj Shand,1269,2013-01-10 11:30:00.0000000,2016-05-24 11:30:00.0000000,0.00
Katie Darwin,1266,2013-01-11 11:30:00.0000000,2016-05-31 11:30:00.0000000,0.00
Hudson Hollinworth,1240,2013-01-22 11:30:00.0000000,2016-05-12 11:30:00.0000000,0.00


Task 5 (Query 5) : Narrow down the suspects and look for any further discrepencies

\- Compares each employee based off of how many customers they've had, their total transactions, their smallest, median, and highest sales, and the biggest sale they've had

\- <span style="color: var(--vscode-foreground);">&nbsp;Joins Application.People and ct.LastEditedBy</span>

\- Focuses on the two most suspicious employees to make the final conclusion

In [102]:
SELECT 
    p.FullName AS Employee,
    COUNT(DISTINCT ct.CustomerID) AS CustomersHandled,
    COUNT(ct.CustomerTransactionID) AS TotalTransactions,
    MIN(ct.TransactionAmount) AS SmallestTransaction,
    AVG(ct.TransactionAmount) AS AvgTransaction,
    MAX(ct.TransactionAmount) AS LargestTransaction,
    SUM(ct.TransactionAmount) AS TotalAmount
FROM Sales.CustomerTransactions ct
INNER JOIN Application.People p ON ct.LastEditedBy = p.PersonID
WHERE p.FullName IN ('Hudson Onslow', 'Anthony Grosse')  -- Put both suspect names here
GROUP BY p.FullName
ORDER BY TotalAmount DESC

(2 rows affected)

Total execution time: 00:00:00.037

Employee,CustomersHandled,TotalTransactions,SmallestTransaction,AvgTransaction,LargestTransaction,TotalAmount
Hudson Onslow,259,5929,-169387.42,45.034818,28280.80,267011.44
Anthony Grosse,257,6100,-140448.53,0.000000,25247.55,0.00


# Mystery #2 - The Cold Case

## The Warehouse was typically a non-eventful area of the company. That was until the Health Inspectors arrived at the Facility's freezer area and found out that the Temperature Logs were tampered with. If the FDA shuts down the Cold Storage, the Company could lose a major part of its revenue stream. You must find out who's behind this before further actions are taken.

Task 1 (Query 1): Find out which room is malfunctioning the most ( The regular temperatures range from -10 and -5 Degrees Celsius)

\- Finds the Room with the Highest temperature fluctuation and displays it

\- Subquery calculates the average temperate per sensor 

\- Identifies which sensor has been tampered with

In [103]:
SELECT TOP 1
    ColdRoomTemperatureID,
    ColdRoomSensorNumber,
    Temperature,
    RecordedWhen,
    (SELECT AVG(Temperature) 
     FROM Warehouse.ColdRoomTemperatures 
     WHERE ColdRoomSensorNumber = crt.ColdRoomSensorNumber) AS AvgTempForSensor
FROM Warehouse.ColdRoomTemperatures crt
WHERE Temperature > 4 OR Temperature < 2
ORDER BY Temperature DESC

(1 row affected)

Total execution time: 00:00:00.015

ColdRoomTemperatureID,ColdRoomSensorNumber,Temperature,RecordedWhen,AvgTempForSensor
3654738,2,4.70,2016-05-31 23:59:24.0000000,4.700000


Task 2 (Query 2): Assess which items would be most affected

\- Joins Warehouse.StockItemHoldings, si.StockItemID, Application.People, and si.LastEditedBy

\- Calculates Total Value Risk on Line 9 and ranks the Highest amounts to the Lowest

\- Can see who would have motive to move these items around

In [104]:
SELECT TOP 20
    si.StockItemID,
    si.StockItemName,
    si.UnitPrice,
    sih.QuantityOnHand,
    (si.UnitPrice * sih.QuantityOnHand) AS TotalValue
FROM Warehouse.StockItems si
INNER JOIN Warehouse.StockItemHoldings sih ON si.StockItemID = sih.StockItemID
WHERE si.IsChillerStock = 1
    AND sih.QuantityOnHand > 0
ORDER BY TotalValue DESC

(8 rows affected)

Total execution time: 00:00:00.010

StockItemID,StockItemName,UnitPrice,QuantityOnHand,TotalValue
227,White chocolate moon rocks 250g,8.55,245352,2097759.60
226,White chocolate snow balls 250g,8.55,155856,1332568.80
225,Chocolate sharks 250g,8.55,119560,1022238.00
220,Novelty chilli chocolates 250g,8.55,90360,772578.00
221,Novelty chilli chocolates 500g,14.50,48228,699306.00
223,Chocolate echidnas 250g,8.55,74256,634888.80
222,Chocolate beetles 250g,8.55,69336,592822.80
224,Chocolate frogs 250g,8.55,67488,577022.40


Task 3 (Query 3): Check whether orders were put through before the problem was discovered and who was moving Chiller stock around

\- <span style="color: var(--vscode-foreground);">Connects</span> <span style="color: var(--vscode-foreground);">orders to their line items and stock details and links them with the customer and sales person involved</span>

<span style="color: var(--vscode-foreground);">- Calculates</span> <span style="color: var(--vscode-foreground);">line totals to identify the biggest sales&nbsp;</span>    

<span style="color: var(--vscode-foreground);">- Focuses on 2016 orders and Chiller Items only</span>

In [105]:
SELECT TOP 20
    o.OrderID,
    o.OrderDate,
    c.CustomerName,
    ol.StockItemID,
    si.StockItemName,
    ol.Quantity,
    ol.UnitPrice,
    (ol.Quantity * ol.UnitPrice) AS LineTotal,
    p.FullName AS Salesperson
FROM Sales.Orders o
INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
INNER JOIN Warehouse.StockItems si ON ol.StockItemID = si.StockItemID
INNER JOIN Sales.Customers c ON o.CustomerID = c.CustomerID
INNER JOIN Application.People p ON o.SalespersonPersonID = p.PersonID
WHERE si.IsChillerStock = 1
    AND YEAR(o.OrderDate) = 2016
ORDER BY LineTotal DESC

(20 rows affected)

Total execution time: 00:00:00.044

OrderID,OrderDate,CustomerName,StockItemID,StockItemName,Quantity,UnitPrice,LineTotal,Salesperson
72387,2016-05-13,Celica Barajas,222,Chocolate beetles 250g,240,8.55,2052.00,Hudson Hollinworth
70003,2016-04-08,"Tailspin Toys (Marcell, MN)",227,White chocolate moon rocks 250g,240,8.55,2052.00,Amy Trefl
68328,2016-03-11,"Tailspin Toys (Boyden Arbor, SC)",225,Chocolate sharks 250g,240,8.55,2052.00,Amy Trefl
66036,2016-02-03,"Tailspin Toys (Hambleton, WV)",222,Chocolate beetles 250g,240,8.55,2052.00,Archer Lamble
67930,2016-03-04,Chompoo Atitarn,224,Chocolate frogs 250g,240,8.55,2052.00,Hudson Onslow
72865,2016-05-20,"Tailspin Toys (Annamoriah, WV)",222,Chocolate beetles 250g,240,8.55,2052.00,Jack Potter
68316,2016-03-11,Celica Barajas,224,Chocolate frogs 250g,240,8.55,2052.00,Anthony Grosse
68114,2016-03-08,"Wingtip Toys (Chaseley, ND)",220,Novelty chilli chocolates 250g,240,8.55,2052.00,Kayla Woodcock
68402,2016-03-14,"Wingtip Toys (Bethel Acres, OK)",220,Novelty chilli chocolates 250g,240,8.55,2052.00,Sophia Hinton
68566,2016-03-17,Gopalgobinda Sikdar,220,Novelty chilli chocolates 250g,240,8.55,2052.00,Jack Potter


Task 4 (Query 4): Find out who's been making transactions for Chiller items

\- Analyzes <span style="color: var(--vscode-foreground);">stock transaction logs to track inventory movements</span>

<span style="color: var(--vscode-foreground);">- Uses a subquery</span> <span style="color: var(--vscode-foreground);">to filter transactions for only chiller stock items</span>

<span style="color: var(--vscode-foreground);">-&nbsp;</span> <span style="color: var(--vscode-foreground);">Shows</span> <span style="color: var(--vscode-foreground);"> the timeframe of each employee's activity and Ranks&nbsp;</span> <span style="color: var(--vscode-foreground);">employees by transaction volume to find unusual patterns</span>

In [106]:
SELECT
    ISNULL(p.FullName, 'Unknown') AS Employee,
    COUNT(DISTINCT sit.StockItemTransactionID) AS TransactionsHandled,
    COUNT(DISTINCT sit.StockItemID) AS DifferentItemsHandled,
    SUM(CASE WHEN sit.Quantity < 0 THEN ABS(sit.Quantity) ELSE 0 END) AS TotalUnitsSold,
    MIN(sit.TransactionOccurredWhen) AS FirstTransaction,
    MAX(sit.TransactionOccurredWhen) AS LastTransaction
FROM Warehouse.StockItemTransactions sit
LEFT JOIN Application.People p ON sit.LastEditedBy = p.PersonID
WHERE sit.StockItemID IN 
    (SELECT StockItemID 
     FROM Warehouse.StockItems 
     WHERE IsChillerStock = 1)
    AND YEAR(sit.TransactionOccurredWhen) = 2016
GROUP BY p.FullName
ORDER BY TransactionsHandled DESC

(19 rows affected)

Total execution time: 00:00:00.039

Employee,TransactionsHandled,DifferentItemsHandled,TotalUnitsSold,FirstTransaction,LastTransaction
Isabella Rupp,83,8,10308.000,2016-01-18 12:00:00.0000000,2016-05-28 12:00:00.0000000
Jai Shand,82,8,9936.000,2016-01-12 12:00:00.0000000,2016-05-23 12:00:00.0000000
Lily Code,79,8,10548.000,2016-01-07 12:00:00.0000000,2016-05-26 12:00:00.0000000
Eva Muirden,79,8,9828.000,2016-01-06 12:00:00.0000000,2016-05-11 12:00:00.0000000
Anthony Grosse,74,8,8880.000,2016-01-23 12:00:00.0000000,2016-05-19 12:00:00.0000000
Hudson Onslow,60,8,7884.000,2016-02-19 12:00:00.0000000,2016-05-31 12:00:00.0000000
Amy Trefl,56,8,6792.000,2016-01-09 12:00:00.0000000,2016-05-27 12:00:00.0000000
Archer Lamble,56,8,7452.000,2016-01-29 12:00:00.0000000,2016-05-07 12:00:00.0000000
Hudson Hollinworth,56,8,6360.000,2016-01-05 07:00:00.0000000,2016-05-02 12:00:00.0000000
Ethan Onslow,56,8,6264.000,2016-01-04 07:00:00.0000000,2016-05-21 12:00:00.0000000


Task 5 (Query 5): Compare the Prime Suspects activity to other Employees to see who's been profitting the most

<span style="color: var(--vscode-foreground);">-&nbsp;&nbsp;</span>  <span style="color: var(--vscode-foreground);">Aggregates</span> <span style="color: var(--vscode-foreground);">all chiller stock orders by salesperson and groups&nbsp;</span> <span style="color: var(--vscode-foreground);">results by employee to compare performance</span>

<span style="color: var(--vscode-foreground);">-&nbsp; Joins&nbsp;</span>  <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Sales.OrderLines, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">o.OrderID, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Application.People, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">o.SalespersonPersonID, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Warehouse.StockItems, and </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ol.StockItemID</span>

<span style="color: var(--vscode-foreground);">-&nbsp;</span>  <span style="color: var(--vscode-foreground);">Ranks </span> <span style="color: var(--vscode-foreground);">salespeople by total revenue generated to expose who's been pushing the most of the products</span>

In [109]:
SELECT TOP 5
    p.FullName AS Employee,
    COUNT(DISTINCT o.OrderID) AS TotalOrders,
    SUM(ol.Quantity) AS TotalUnitsOrdered,
    AVG(ol.UnitPrice) AS AvgPricePerUnit,
    SUM(ol.Quantity * ol.UnitPrice) AS TotalRevenue
FROM Sales.Orders o
INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
INNER JOIN Application.People p ON o.SalespersonPersonID = p.PersonID
INNER JOIN Warehouse.StockItems si ON ol.StockItemID = si.StockItemID
WHERE si.IsChillerStock = 1
    AND YEAR(o.OrderDate) = 2016
GROUP BY p.FullName
ORDER BY TotalRevenue DESC

(5 rows affected)

Total execution time: 00:00:00.027

Employee,TotalOrders,TotalUnitsOrdered,AvgPricePerUnit,TotalRevenue
Hudson Hollinworth,110,13920,9.385087,126013.20
Taj Shand,105,13596,9.718750,125742.00
Sophia Hinton,102,13608,9.259633,121632.00
Archer Lamble,101,13128,9.243203,118242.00
Amy Trefl,109,13020,9.346875,116533.20


# Mystery #3 - W.W. Cooper

## Recently, an order was made with the invoice being paid off, the order leaving the Warehouse, and the Driver making his way towards his destination. However, the shipment and the driver never arrived and disappeared without a trace. It is believed that this was an inside job. It's your job to scrounge through the database and piece together a trail of how they got away

Task 1 (Query 1) - Find the largest, most valuable invoices from 2016, one of them is the target

<span style="color: var(--vscode-foreground);">- Joins invoices with their line items to calculate total shipment valued</span>

<span style="color: var(--vscode-foreground);">- Calculates both profit and total value</span>

<span style="color: var(--vscode-foreground);">- Identifies who was involved these high-value transactions, CustomerID is used in Task 2</span>

In [110]:
SELECT TOP 20
    i.InvoiceID,
    i.CustomerID,
    i.InvoiceDate,
    i.LastEditedBy,
    COUNT(il.InvoiceLineID) AS NumberOfItems,
    SUM(il.LineProfit) AS TotalProfit,
    SUM(il.ExtendedPrice) AS TotalValue
FROM Sales.Invoices i
INNER JOIN Sales.InvoiceLines il ON i.InvoiceID = il.InvoiceID
WHERE YEAR(i.InvoiceDate) = 2016
GROUP BY i.InvoiceID, i.CustomerID, i.InvoiceDate, i.LastEditedBy
ORDER BY TotalValue DESC

(20 rows affected)

Total execution time: 00:00:00.192

InvoiceID,CustomerID,InvoiceDate,LastEditedBy,NumberOfItems,TotalProfit,TotalValue
64220,71,2016-02-20,16,5,12092.00,33966.40
67035,1043,2016-04-07,14,3,11590.50,31489.30
69043,415,2016-05-07,7,5,11472.00,31470.90
70445,492,2016-05-31,3,5,10605.00,28280.80
65919,67,2016-03-21,11,4,9849.60,27219.12
61812,1056,2016-01-07,19,4,9110.40,25334.94
62060,42,2016-01-12,4,4,9259.50,25247.55
69291,498,2016-05-12,10,5,8939.00,24735.80
63360,983,2016-02-03,15,5,9145.00,24110.90
68978,904,2016-05-06,17,3,8871.00,23877.45


Task 2 (Query 2) - View the original order and see who was involved with it

\- Searches for orders matching Customer of Interest

\- Reveals who sold and who picked the items (Suspects) and the timing of the order

\- Use CustomerID and OrderDate from last Task to find the suspected order

In [111]:
SELECT 
    OrderID,
    CustomerID,
    OrderDate,
    ExpectedDeliveryDate,
    PickingCompletedWhen,
    SalespersonPersonID,
    PickedByPersonID,
    ContactPersonID,
    LastEditedBy,
    LastEditedWhen,
    DATEDIFF(day, OrderDate, PickingCompletedWhen) AS DaysToPickOrder
FROM Sales.Orders
WHERE CustomerID = 71 -- Found in Task 1
    AND YEAR(OrderDate) = 2016
ORDER BY OrderDate DESC

(27 rows affected)

Total execution time: 00:00:00.007

OrderID,CustomerID,OrderDate,ExpectedDeliveryDate,PickingCompletedWhen,SalespersonPersonID,PickedByPersonID,ContactPersonID,LastEditedBy,LastEditedWhen,DaysToPickOrder
73457,71,2016-05-30,2016-05-31,2016-05-30 11:00:00.0000000,15,5,1141,5,2016-05-30 11:00:00.0000000,0
71699,71,2016-05-04,2016-05-05,2016-05-04 11:00:00.0000000,3,15,1141,15,2016-05-04 11:00:00.0000000,0
71641,71,2016-05-03,2016-05-04,2016-05-03 11:00:00.0000000,14,20,1141,20,2016-05-03 11:00:00.0000000,0
70972,71,2016-04-22,2016-04-25,2016-04-22 12:00:00.0000000,20,NULL,1141,14,2016-04-22 12:00:00.0000000,0
71021,71,2016-04-22,2016-04-25,2016-04-26 11:00:00.0000000,20,13,1141,13,2016-04-26 11:00:00.0000000,4
70139,71,2016-04-11,2016-04-12,2016-04-11 12:00:00.0000000,20,NULL,1141,10,2016-04-11 12:00:00.0000000,0
70198,71,2016-04-11,2016-04-12,NULL,20,NULL,1141,10,2016-04-11 12:00:00.0000000,NULL
69902,71,2016-04-07,2016-04-08,2016-04-07 11:00:00.0000000,14,19,1141,19,2016-04-07 11:00:00.0000000,0
69925,71,2016-04-07,2016-04-08,2016-04-07 11:00:00.0000000,20,19,1141,19,2016-04-07 11:00:00.0000000,0
69348,71,2016-03-29,2016-03-30,2016-03-29 11:00:00.0000000,7,7,1141,7,2016-03-29 11:00:00.0000000,0


Task 3 (Query 3) - Investigate the Customer Profile of the original order

\- Displays complete information about the customer

\- Lists possible red flags, possible leading to the conclusion that the customer is a front

\- Shows who was involved, showing a connection between suspects

In [112]:
SELECT 
    CustomerID,
    CustomerName,
    CreditLimit,
    AccountOpenedDate,
    PaymentDays,
    PhoneNumber,
    DeliveryAddressLine1,
    DeliveryAddressLine2,
    PostalAddressLine1,
    PostalAddressLine2,
    PrimaryContactPersonID,
    AlternateContactPersonID,
    DeliveryCityID,
    PostalCityID,
    LastEditedBy,
    ValidFrom
FROM Sales.Customers
WHERE CustomerID = 71

(1 row affected)

Total execution time: 00:00:00.010

CustomerID,CustomerName,CreditLimit,AccountOpenedDate,PaymentDays,PhoneNumber,DeliveryAddressLine1,DeliveryAddressLine2,PostalAddressLine1,PostalAddressLine2,PrimaryContactPersonID,AlternateContactPersonID,DeliveryCityID,PostalCityID,LastEditedBy,ValidFrom
71,"Tailspin Toys (Good Hart, MI)",NULL,2013-01-01,7,(231) 555-0100,Shop 80,2001 Rajanala Road,PO Box 9785,Espositoville,1141,1142,13375,13375,1,2013-01-01 00:00:00.0000000


Task 4 (Query 4) - Cross-References Everyone involved

\- Lists all employees and buyers who are connected in the case

\- Shows their roles, contact information, and permissions

\- Identifies when they were hired

In [113]:
SELECT 
    PersonID,
    FullName,
    PreferredName,
    IsEmployee,
    IsSalesperson,
    PhoneNumber,
    EmailAddress,
    IsPermittedToLogon,
    ValidFrom,
    ValidTo
FROM Application.People
WHERE PersonID IN (16, 9, 1141, 1142)  -- From SalespersonID, PickedByPersonID, & ContactPersonID From Previous Query Searches
ORDER BY PersonID

(4 rows affected)

Total execution time: 00:00:00.005

PersonID,FullName,PreferredName,IsEmployee,IsSalesperson,PhoneNumber,EmailAddress,IsPermittedToLogon,ValidFrom,ValidTo
9,Alica Fatnowna,Alica,1,0,(415) 555-0102,alicaf@wideworldimporters.com,1,2016-05-31 23:14:00.0000000,9999-12-31 23:59:59.9999999
16,Archer Lamble,Archer,1,1,(415) 555-0102,archerl@wideworldimporters.com,0,2016-05-31 23:14:00.0000000,9999-12-31 23:59:59.9999999
1141,Afshin Djalili,Afshin,0,0,(231) 555-0100,afshin@tailspintoys.com,0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
1142,Vaclav Holy,Vaclav,0,0,(231) 555-0100,vaclav@tailspintoys.com,0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


Task 5 (Query 5) - Display the original order details to see if it was paid or unpaid

\- Tracks finances of transaction

\- Shows invoice amount, payments received, and any outstanding balances

\- Displays when transactions were finalized

In [114]:
SELECT 
    CustomerTransactionID,
    TransactionTypeID,
    InvoiceID,
    TransactionAmount,
    TransactionDate,
    OutstandingBalance,
    FinalizationDate,
    LastEditedBy,
    LastEditedWhen
FROM Sales.CustomerTransactions
WHERE InvoiceID = 64220
ORDER BY TransactionDate DESC

(1 row affected)

Total execution time: 00:00:00.006

CustomerTransactionID,TransactionTypeID,InvoiceID,TransactionAmount,TransactionDate,OutstandingBalance,FinalizationDate,LastEditedBy,LastEditedWhen
306399,1,64220,33966.40,2016-02-20,0.00,2016-02-21,20,2016-02-21 11:30:00.0000000


# Mystery #4 - Insider Shipping

## There's been somebody within the company who has been shipping expensive items to a dedicated group of customers giving them priority over others. Find out who this Employee is and who their Associated are from the general customer list

Task 1 (Query 1) - Finds the person with the most picked orders

\- Differentiates who's making and picking these orders based off of their Individual IDs

\- Displays volume of orders placed to sort out Discrepencies

\- Shows First and Last pick times

In [115]:
SELECT TOP 5
    PickedByPersonID,
    COUNT(OrderID) AS OrdersPicked,
    MIN(PickingCompletedWhen) AS FirstPick,
    MAX(PickingCompletedWhen) AS LastPick
FROM Sales.Orders
WHERE PickingCompletedWhen IS NOT NULL
    AND PickedByPersonID IS NOT NULL
    AND YEAR(PickingCompletedWhen) = 2016
GROUP BY PickedByPersonID
ORDER BY OrdersPicked DESC

(5 rows affected)

Total execution time: 00:00:00.038

PickedByPersonID,OrdersPicked,FirstPick,LastPick
17,983,2016-01-04 11:00:00.0000000,2016-05-14 11:00:00.0000000
8,767,2016-01-19 11:00:00.0000000,2016-05-26 11:00:00.0000000
7,738,2016-01-14 11:00:00.0000000,2016-05-25 11:00:00.0000000
11,669,2016-01-01 11:00:00.0000000,2016-05-24 11:00:00.0000000
10,633,2016-01-05 11:00:00.0000000,2016-05-18 11:00:00.0000000


Task 2 (Query 2) - <span style="color: var(--vscode-foreground);">Finds customers who receive a suspicious number of orders from the suspected picker</span>

<span style="color: var(--vscode-foreground);">- Counts how many orders each customer received from the suspect and displays the top ones via their Customer ID</span>

<span style="color: var(--vscode-foreground);">- Uses UNION to find buyers who have more than 3 orders from the suspect or orders that exceed 10,000 dollars with the suspect</span>

<span style="color: var(--vscode-foreground);">- Reveals potential conspirators</span>

In [ ]:
SELECT DISTINCT CustomerID -- Customers who got more than 3 orders from suspect
FROM Sales.Orders
WHERE PickedByPersonID = 17
    AND CustomerID IN (
        SELECT CustomerID
        FROM Sales.Orders
        WHERE PickedByPersonID = 17
        GROUP BY CustomerID
        HAVING COUNT(*) >= 3
    )
UNION
SELECT DISTINCT o.CustomerID -- Customers with over 10000 in sales with suspect
FROM Sales.Orders o
INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
WHERE o.PickedByPersonID = 17
GROUP BY o.CustomerID
HAVING SUM(ol.Quantity * ol.UnitPrice) > 10000

Task 3 (Query 3) - Find which high-value stock items appear in the suspect's orders.

\- <span style="color: var(--vscode-foreground);">Uses INTERSECT</span> <span style="color: var(--vscode-foreground);">to find items that are both expensive and picked by the suspect</span>

<span style="color: var(--vscode-foreground);">- Shows the overlap between high-value items and suspect's activity</span>

<span style="color: var(--vscode-foreground);">- Identifies&nbsp;</span> <span style="color: var(--vscode-foreground);">which premium goods are being stolen</span>

In [117]:
SELECT DISTINCT -- Items over $50
    StockItemID
FROM Warehouse.StockItems
WHERE UnitPrice > 50
INTERSECT
SELECT DISTINCT -- Items picked by the suspect
    ol.StockItemID
FROM Sales.Orders o
INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
WHERE o.PickedByPersonID = 17;

(23 rows affected)

Total execution time: 00:00:00.026

StockItemID
8
15
67
68
69
70
71
72
73
74


Task 4 (Query 4) - Check if the accomplice customers actually paid for their orders.

<span style="color: var(--vscode-foreground);">- S</span><span style="color: var(--vscode-foreground);">hows</span> <span style="color: var(--vscode-foreground);"> if payments were made or if goods left unpaid</span>

<span style="color: var(--vscode-foreground);">- Reveals accomplices in either embezzlement or theft</span>

In [118]:
SELECT 
    CustomerID,
    COUNT(CustomerTransactionID) AS TotalTransactions,
    SUM(TransactionAmount) AS TotalAmount,
    SUM(OutstandingBalance) AS StillOwed
FROM Sales.CustomerTransactions
WHERE CustomerID IN (86, 850, 68, 970, 442)  -- Uses CustomerID's from Query 2
GROUP BY CustomerID
ORDER BY StillOwed DESC

(2 rows affected)

Total execution time: 00:00:00.030

CustomerID,TotalTransactions,TotalAmount,StillOwed
850,230,0.00,0.00
970,208,0.00,0.00


Task 5 (Query 5) - Reveal the Suspect  
<span style="color: var(--vscode-foreground);">-&nbsp;</span> <span style="color: var(--vscode-foreground);">Looks up the employee information</span>  
\- Shows their identity and personal information  
\- Provides Access Level

In [119]:
SELECT 
    PersonID,
    FullName,
    PreferredName,
    IsEmployee,
    IsSalesperson,
    PhoneNumber,
    EmailAddress
FROM Application.People
WHERE PersonID = 17

(1 row affected)

Total execution time: 00:00:00.005

PersonID,FullName,PreferredName,IsEmployee,IsSalesperson,PhoneNumber,EmailAddress
17,Piper Koch,Piper,1,0,(415) 555-0102,piperk@wideworldimporters.com


# Mystery # 5 - Playing Favorites

## The Accounting Team had noticed something odd, some customers were getting lower prices on items while others were getting charged an insane amount for the same item. This fluctuation was costing the Company a large sum in sales. Its your Job to figure out who's been doing this and which items they've been tampering with

Task 1 (Query 1) - Find which stock items have the biggest price variations across different sales.

\- Calculates Price Statistics via a CTE

\- Shows Minimum, Maximum, and Average Prices Charged

\- Reveals items with price inconsistencies

In [120]:
WITH PriceAnalysis AS (
    SELECT 
        StockItemID,
        MIN(UnitPrice) AS LowestPrice,
        MAX(UnitPrice) AS HighestPrice,
        AVG(UnitPrice) AS AveragePrice,
        COUNT(DISTINCT UnitPrice) AS DifferentPrices
    FROM Sales.OrderLines
    WHERE YEAR(PickingCompletedWhen) = 2016
    GROUP BY StockItemID
)
SELECT TOP 15
    pa.StockItemID,
    si.StockItemName,
    pa.LowestPrice,
    pa.HighestPrice,
    pa.AveragePrice,
    (pa.HighestPrice - pa.LowestPrice) AS PriceRange,
    pa.DifferentPrices
FROM PriceAnalysis pa
INNER JOIN Warehouse.StockItems si ON pa.StockItemID = si.StockItemID
WHERE pa.HighestPrice > pa.LowestPrice
ORDER BY PriceRange DESC

(14 rows affected)

Total execution time: 00:00:00.100

StockItemID,StockItemName,LowestPrice,HighestPrice,AveragePrice,PriceRange,DifferentPrices
15,USB food flash drive - dessert 10 drive variety pack,24.00,240.00,169.125000,216.00,3
8,USB food flash drive - dim sum 10 drive variety pack,24.00,240.00,178.717557,216.00,3
10,USB food flash drive - chocolate bar,3.20,32.00,22.386666,28.80,3
5,USB food flash drive - hamburger,3.20,32.00,25.819847,28.80,3
7,USB food flash drive - pizza slice,3.20,32.00,24.650847,28.80,3
11,USB food flash drive - cookie,3.20,32.00,25.775000,28.80,3
9,USB food flash drive - banana,3.20,32.00,24.025000,28.80,3
6,USB food flash drive - hot dog,3.20,32.00,23.174193,28.80,3
13,USB food flash drive - shrimp cocktail,3.20,32.00,23.771428,28.80,3
4,USB food flash drive - sushi roll,3.20,32.00,22.482352,28.80,3


Task 2 (Query 2) - Finds Customers who're paying suspicously low for items

\- Uses a CTE to calculate the average price per Customer

\- Uses the suspicous StockItemID from Task 1 to identify who's been paying less

\- Shows how low the prices are being charged for as opposed to the average prices

In [121]:
WITH CustomerPricing AS (
    SELECT 
        o.CustomerID,
        AVG(ol.UnitPrice) AS AvgPricePaid,
        COUNT(ol.OrderLineID) AS TimesPurchased,
        SUM(ol.Quantity) AS TotalQuantity
    FROM Sales.Orders o
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    WHERE ol.StockItemID = 15 -- Uses StockItemID from Query 1
        AND YEAR(o.OrderDate) = 2016
    GROUP BY o.CustomerID
)
SELECT 
    cp.CustomerID,
    c.CustomerName,
    cp.AvgPricePaid,
    cp.TimesPurchased,
    cp.TotalQuantity,
    (SELECT AVG(UnitPrice) FROM Sales.OrderLines WHERE StockItemID = 15) AS MarketAverage
FROM CustomerPricing cp
INNER JOIN Sales.Customers c ON cp.CustomerID = c.CustomerID
ORDER BY cp.AvgPricePaid ASC

(111 rows affected)

Total execution time: 00:00:00.053

CustomerID,CustomerName,AvgPricePaid,TimesPurchased,TotalQuantity,MarketAverage
599,"Wingtip Toys (Dickworsham, TX)",24.000000,1,7,231.481690
594,"Wingtip Toys (Marfa, TX)",24.000000,1,8,231.481690
556,"Wingtip Toys (Morita, TX)",24.000000,1,2,231.481690
554,"Wingtip Toys (Hollandsburg, IN)",24.000000,1,1,231.481690
549,"Wingtip Toys (Lake Ronkonkoma, NY)",24.000000,1,2,231.481690
547,"Wingtip Toys (Chaseley, ND)",24.000000,1,10,231.481690
525,"Wingtip Toys (Claycomo, MO)",24.000000,1,7,231.481690
517,"Wingtip Toys (Licking, MO)",24.000000,1,7,231.481690
515,"Wingtip Toys (Del Valle, TX)",24.000000,1,5,231.481690
494,"Wingtip Toys (Pikeview, CO)",24.000000,1,2,231.481690


Task 3 (Query 3) - <span style="color: var(--vscode-foreground);">Find which salesperson is handling out these fraudulent orders.</span>

<span style="color: var(--vscode-foreground);">-&nbsp;</span> <span style="color: var(--vscode-foreground);">Uses a CTE</span> <span style="color: var(--vscode-foreground);"> to track orders by salesperson for the suspect item and c</span><span style="color: var(--vscode-foreground);">ounts</span> <span style="color: var(--vscode-foreground);"> how many discount orders each salesperson processed&nbsp;</span> 

<span style="color: var(--vscode-foreground);">- Reveals Employee who's responsible</span>

<span style="color: var(--vscode-foreground);">- Calculates the average discount</span>

In [122]:
WITH SalespersonActivity AS (
    SELECT 
        o.SalespersonPersonID,
        COUNT(DISTINCT o.OrderID) AS OrdersProcessed,
        AVG(ol.UnitPrice) AS AvgPriceCharged,
        COUNT(DISTINCT o.CustomerID) AS DifferentCustomers
    FROM Sales.Orders o
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    WHERE ol.StockItemID = 15 -- Uses StockItemID from Previous Query
        AND YEAR(o.OrderDate) = 2016
    GROUP BY o.SalespersonPersonID
)
SELECT 
    sa.SalespersonPersonID,
    p.FullName,
    sa.OrdersProcessed,
    sa.AvgPriceCharged,
    sa.DifferentCustomers,
    (SELECT AVG(UnitPrice) FROM Sales.OrderLines WHERE StockItemID = 15) AS MarketAverage
FROM SalespersonActivity sa
INNER JOIN Application.People p ON sa.SalespersonPersonID = p.PersonID
ORDER BY sa.AvgPriceCharged ASC

(10 rows affected)

Total execution time: 00:00:00.046

SalespersonPersonID,FullName,OrdersProcessed,AvgPriceCharged,DifferentCustomers,MarketAverage
3,Hudson Onslow,15,113.600000,15,231.481690
13,Hudson Hollinworth,10,157.200000,10,231.481690
20,Jack Potter,19,162.315789,19,231.481690
15,Taj Shand,17,163.764705,16,231.481690
16,Archer Lamble,11,164.727272,11,231.481690
2,Kayla Woodcock,7,180.000000,7,231.481690
14,Lily Code,11,181.090909,11,231.481690
6,Sophia Hinton,11,183.272727,11,231.481690
8,Anthony Grosse,14,194.571428,14,231.481690
7,Amy Trefl,13,207.692307,13,231.481690


Task 4 (Query 4) - Calculate Losses from Fraud

\- <span style="color: var(--vscode-foreground);">Compares</span> <span style="color: var(--vscode-foreground);"> actual prices vs. market prices</span>

\- <span style="color: var(--vscode-foreground);">Calculates</span> <span style="color: var(--vscode-foreground);"> total lost revenue</span>

\- <span style="color: var(--vscode-foreground);">Shows</span> <span style="color: var(--vscode-foreground);"> the financial impact per customer</span>

In [123]:
WITH LossCalculation AS (
    SELECT 
        o.CustomerID,
        ol.Quantity,
        ol.UnitPrice AS ActualPrice,
        (SELECT AVG(UnitPrice) FROM Sales.OrderLines WHERE StockItemID = 15) AS MarketPrice,
        ol.Quantity * ol.UnitPrice AS ActualRevenue,
        ol.Quantity * (SELECT AVG(UnitPrice) FROM Sales.OrderLines WHERE StockItemID = 15) AS ExpectedRevenue
    FROM Sales.Orders o
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    WHERE ol.StockItemID = 15 -- Uses StockItemID from previous Query
        AND o.SalespersonPersonID = 3 -- Uses SalespersonPersonID from previous Query
        AND YEAR(o.OrderDate) = 2016
)
SELECT 
    lc.CustomerID,
    c.CustomerName,
    SUM(lc.Quantity) AS TotalUnits,
    SUM(lc.ActualRevenue) AS ActualRevenue,
    SUM(lc.ExpectedRevenue) AS ExpectedRevenue,
    SUM(lc.ExpectedRevenue - lc.ActualRevenue) AS TotalLoss
FROM LossCalculation lc
INNER JOIN Sales.Customers c ON lc.CustomerID = c.CustomerID
GROUP BY lc.CustomerID, c.CustomerName
ORDER BY TotalLoss DESC

(15 rows affected)

Total execution time: 00:00:00.037

CustomerID,CustomerName,TotalUnits,ActualRevenue,ExpectedRevenue,TotalLoss
420,"Wingtip Toys (Herlong, CA)",10,240.00,2314.816900,2074.816900
547,"Wingtip Toys (Chaseley, ND)",10,240.00,2314.816900,2074.816900
564,"Wingtip Toys (Tilleda, WI)",9,216.00,2083.335210,1867.335210
102,"Tailspin Toys (Fieldbrook, CA)",8,288.00,1851.853520,1563.853520
525,"Wingtip Toys (Claycomo, MO)",7,168.00,1620.371830,1452.371830
82,"Tailspin Toys (La Cueva, NM)",7,252.00,1620.371830,1368.371830
138,"Tailspin Toys (Navassa, NC)",6,216.00,1388.890140,1172.890140
482,"Wingtip Toys (Caton, NY)",2,48.00,462.963380,414.963380
175,"Tailspin Toys (San Acacia, NM)",1,36.00,231.481690,195.481690
559,"Wingtip Toys (Mendoza, TX)",1,240.00,231.481690,-8.518310


Task 5 (Query 5) - <span style="color: var(--vscode-foreground);">Check if this salesperson is manipulating prices on other items too</span>

<span style="color: var(--vscode-foreground);">- Identifies</span> <span style="color: var(--vscode-foreground);">other items that are being sold at suspicious discounts</span>

<span style="color: var(--vscode-foreground);">- Shows</span> <span style="color: var(--vscode-foreground);">the full scope of the fraud</span>

<span style="color: var(--vscode-foreground);">- Reveals</span> <span style="color: var(--vscode-foreground);">if this is systematic or isolated</span>

In [124]:
WITH SuspectPricing AS (
    SELECT 
        ol.StockItemID,
        AVG(ol.UnitPrice) AS SuspectAvgPrice,
        COUNT(DISTINCT o.OrderID) AS SuspectOrders
    FROM Sales.Orders o
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    WHERE o.SalespersonPersonID = 3
        AND YEAR(o.OrderDate) = 2016
    GROUP BY ol.StockItemID
),
MarketPricing AS (
    SELECT 
        ol.StockItemID,
        AVG(ol.UnitPrice) AS MarketAvgPrice
    FROM Sales.Orders o
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    WHERE o.SalespersonPersonID != 3
        AND YEAR(o.OrderDate) = 2016
    GROUP BY ol.StockItemID
)
SELECT TOP 10
    si.StockItemName,
    sp.SuspectAvgPrice,
    mp.MarketAvgPrice,
    (mp.MarketAvgPrice - sp.SuspectAvgPrice) AS Discount,
    sp.SuspectOrders
FROM SuspectPricing sp
INNER JOIN MarketPricing mp ON sp.StockItemID = mp.StockItemID
INNER JOIN Warehouse.StockItems si ON sp.StockItemID = si.StockItemID
WHERE sp.SuspectAvgPrice < mp.MarketAvgPrice
ORDER BY Discount DESC

(5 rows affected)

Total execution time: 00:00:00.114

StockItemName,SuspectAvgPrice,MarketAvgPrice,Discount,SuspectOrders
USB food flash drive - dessert 10 drive variety pack,113.600000,176.495575,62.895575,15
USB food flash drive - pizza slice,20.480000,24.835398,4.355398,5
USB food flash drive - fortune cookie,24.673684,26.922314,2.248630,19
USB food flash drive - shrimp cocktail,22.720000,23.897600,1.177600,15
USB food flash drive - hot dog,22.933333,23.193043,0.259710,9


# Mystery #6- The Potential Scalper

## A large portion of sales are coming from one customer was ordering massive quantities of the same items, far beyond normal business needs. Are they a legitimate bulk buyer, or is an employee creating fake orders to steal inventory for their own profit?

Task 1 (Query 1) - Find the  customer with the most orders

In [140]:
SELECT TOP 10
    CustomerID,
    COUNT(OrderID) AS TotalOrders,
    MIN(OrderDate) AS FirstOrder,
    MAX(OrderDate) AS LastOrder,
    DATEDIFF(day, MIN(OrderDate), MAX(OrderDate)) AS DaysAsCustomer
FROM Sales.Orders
WHERE YEAR(OrderDate) = 2016
GROUP BY CustomerID
ORDER BY TotalOrders DESC

(10 rows affected)

Total execution time: 00:00:00.033

CustomerID,TotalOrders,FirstOrder,LastOrder,DaysAsCustomer
460,30,2016-01-01,2016-05-21,141
950,28,2016-01-04,2016-05-26,143
42,27,2016-01-07,2016-05-30,144
71,27,2016-01-01,2016-05-30,150
183,27,2016-01-04,2016-05-27,144
506,27,2016-01-04,2016-05-26,143
185,26,2016-01-07,2016-05-30,144
37,26,2016-01-04,2016-05-19,136
874,25,2016-01-08,2016-05-30,143
153,25,2016-01-05,2016-05-30,146


Task 2 (Query 2) - Get Full Details on suspicious customer

In [142]:
SELECT 
    CustomerID,
    CustomerName,
    CreditLimit,
    AccountOpenedDate,
    PaymentDays,
    PhoneNumber,
    DeliveryAddressLine1,
    DeliveryAddressLine2,
    PrimaryContactPersonID,
    BuyingGroupID,
    CustomerCategoryID
FROM Sales.Customers
WHERE CustomerID = 460 -- Found from previous Query

(1 row affected)

Total execution time: 00:00:00.020

CustomerID,CustomerName,CreditLimit,AccountOpenedDate,PaymentDays,PhoneNumber,DeliveryAddressLine1,DeliveryAddressLine2,PrimaryContactPersonID,BuyingGroupID,CustomerCategoryID
460,"Wingtip Toys (Mayhill, NM)",NULL,2013-01-01,7,(505) 555-0100,Suite 11,898 Kim Boulevard,2119,2,3


Task 3 (Query 3) - Find the Suspicious Customer;s most frequently ordered

In [143]:
SELECT TOP 15
    ol.StockItemID,
    COUNT(DISTINCT o.OrderID) AS TimesOrdered,
    SUM(ol.Quantity) AS TotalQuantity,
    AVG(ol.UnitPrice) AS AvgPrice,
    SUM(ol.Quantity * ol.UnitPrice) AS TotalSpent
FROM Sales.Orders o
INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
WHERE o.CustomerID = 460
    AND YEAR(o.OrderDate) = 2016
GROUP BY ol.StockItemID
ORDER BY TotalQuantity DESC

(15 rows affected)

Total execution time: 00:00:00.032

StockItemID,TimesOrdered,TotalQuantity,AvgPrice,TotalSpent
187,1,250,0.950000,237.50
194,1,240,4.100000,984.00
180,1,200,1.110000,222.00
179,1,200,1.050000,210.00
178,1,200,2.550000,510.00
199,1,192,3.700000,710.40
193,1,168,3.700000,621.60
222,1,168,8.550000,1436.40
190,1,160,3.500000,560.00
224,1,144,8.550000,1231.20


Task 4 (Query 4) - Get Details on the most - ordered items

In [145]:
SELECT 
    StockItemID,
    StockItemName,
    UnitPrice,
    Brand,
    Size,
    TypicalWeightPerUnit,
    IsChillerStock,
    QuantityPerOuter,
    RecommendedRetailPrice
FROM Warehouse.StockItems
WHERE StockItemID IN (187, 194, 180)  -- Replace with top 3 StockItemIDs from Query 3

(3 rows affected)

Total execution time: 00:00:00.018

StockItemID,StockItemName,UnitPrice,Brand,Size,TypicalWeightPerUnit,IsChillerStock,QuantityPerOuter,RecommendedRetailPrice
180,Shipping carton (Brown) 279x254x217mm,1.11,NULL,279x254x217mm,0.300,0,25,1.66
187,Express post box 5kg (White) 350x280x130mm,0.95,NULL,350x280x130mm,0.200,0,25,1.42
194,Black and orange glass with care despatch tape 48mmx100m,4.10,NULL,48mmx100m,0.700,0,24,6.13


Task 5 (Query 5)- Calculate Total Invoiced Amount to see if they were actually paid for

In [152]:
SELECT 
    i.CustomerID,
    COUNT(DISTINCT i.InvoiceID) AS TotalInvoices,
    SUM(il.ExtendedPrice) AS TotalInvoiced,
    SUM(il.LineProfit) AS TotalProfit
FROM Sales.Invoices i
INNER JOIN Sales.InvoiceLines il ON i.InvoiceID = il.InvoiceID
WHERE i.CustomerID = 460
    AND YEAR(i.InvoiceDate) = 2016
GROUP BY i.CustomerID

(1 row affected)

Total execution time: 00:00:00.024

CustomerID,TotalInvoices,TotalInvoiced,TotalProfit
460,27,62651.74,25958.50


# Mystery #7 - Detours of Deception

### The Logistics Manager noticed something strange, certain delivery routes were taking twice as long as they should, fuel costs were skyrocketing, and customers were complaining about late arrivals. But the delivery confirmations showed everything was "on time.". It seems that somebody has been manipulating the records. Find out who's at the bottom of this

Task 1 (Query 1) - Find all Delivery Methods and who's edited them

In [165]:
SELECT 
    DeliveryMethodID,
    DeliveryMethodName,
    LastEditedBy,
    ValidFrom,
    ValidTo
FROM Application.DeliveryMethods
WHERE ValidTo > '2016-01-01'
ORDER BY DeliveryMethodID

(10 rows affected)

Total execution time: 00:00:00.004

DeliveryMethodID,DeliveryMethodName,LastEditedBy,ValidFrom,ValidTo
1,Post,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
2,Courier,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
3,Delivery Van,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
4,Customer Collect,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
5,Chilled Van,16,2015-01-01 16:00:00.0000000,9999-12-31 23:59:59.9999999
6,Customer Courier to Collect,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
7,Road Freight,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
8,Air Freight,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
9,Refrigerated Road Freight,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
10,Refrigerated Air Freight,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


Task 2 (Query2) - Find the Customer with the most deliveries

In [160]:
SELECT TOP 10
    CustomerID,
    COUNT(OrderID) AS TotalDeliveries,
    MIN(OrderDate) AS FirstDelivery,
    MAX(OrderDate) AS LastDelivery,
    COUNT(DISTINCT DATEPART(MONTH, OrderDate)) AS MonthsActive,
    COUNT(DISTINCT PickedByPersonID) AS DifferentPickers
FROM Sales.Orders
WHERE YEAR(OrderDate) = 2016
    AND PickingCompletedWhen IS NOT NULL
GROUP BY CustomerID
ORDER BY TotalDeliveries DESC

Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:00.059

CustomerID,TotalDeliveries,FirstDelivery,LastDelivery,MonthsActive,DifferentPickers
460,27,2016-01-01,2016-05-21,5,10
506,26,2016-01-04,2016-05-26,5,10
42,24,2016-01-07,2016-05-30,5,12
185,24,2016-01-07,2016-05-30,5,11
950,24,2016-01-04,2016-05-26,5,14
117,24,2016-01-04,2016-05-27,5,13
183,24,2016-01-04,2016-05-27,5,13
37,24,2016-01-04,2016-05-19,5,13
71,23,2016-01-01,2016-05-30,5,12
558,23,2016-02-01,2016-05-27,4,13


Task 3 (Query 3) - Check delivery details for the suspicious customer

In [163]:
SELECT 
    CustomerID,
    CustomerName,
    DeliveryAddressLine1,
    DeliveryAddressLine2,
    DeliveryCityID,
    DeliveryPostalCode,
    PostalAddressLine1,
    PostalAddressLine2,
    PostalCityID,
    PostalPostalCode,
    PhoneNumber,
    PrimaryContactPersonID
FROM Sales.Customers
WHERE CustomerID = 460 -- Uses CustomerID from Query 2

(1 row affected)

Total execution time: 00:00:00.011

CustomerID,CustomerName,DeliveryAddressLine1,DeliveryAddressLine2,DeliveryCityID,DeliveryPostalCode,PostalAddressLine1,PostalAddressLine2,PostalCityID,PostalPostalCode,PhoneNumber,PrimaryContactPersonID
460,"Wingtip Toys (Mayhill, NM)",Suite 11,898 Kim Boulevard,21351,90741,PO Box 2199,Pendyalaville,21351,90741,(505) 555-0100,2119


Task 4 (Query 4) - Get Location Details and Employee Connections

In [162]:
SELECT 
    c.CityName AS DeliveryCity,
    sp.StateProvinceName AS DeliveryState,
    sp.StateProvinceCode,
    c.LatestRecordedPopulation,
    p.PersonID AS ContactPersonID,
    p.FullName AS ContactName,
    p.IsEmployee AS ContactIsEmployee,
    p.PhoneNumber AS ContactPhone
FROM Application.Cities c
INNER JOIN Application.StateProvinces sp ON c.StateProvinceID = sp.StateProvinceID
CROSS JOIN Application.People p
WHERE c.CityID = 21351 -- Uses DeliveryCityID From Query 3
    AND p.PersonID = 2119; -- Uses PrimaryContactPersonId From Query 3

(1 row affected)

Total execution time: 00:00:00.006

DeliveryCity,DeliveryState,StateProvinceCode,LatestRecordedPopulation,ContactPersonID,ContactName,ContactIsEmployee,ContactPhone
Mayhill,New Mexico,NM,75,2119,Daniela Dumina,0,(505) 555-0100


Task 5 (Query 5) - Find out Who's the prime suspect

In [164]:
SELECT 
    o.PickedByPersonID,
    p.FullName AS PickerName,
    p.IsEmployee AS PickerIsEmployee,
    COUNT(o.OrderID) AS OrdersPicked,
    MIN(o.PickingCompletedWhen) AS FirstPick,
    MAX(o.PickingCompletedWhen) AS LastPick
FROM Sales.Orders o
INNER JOIN Application.People p ON o.PickedByPersonID = p.PersonID
WHERE o.CustomerID = 460 -- Uses CustomerID From Previous Queries
    AND o.PickedByPersonID IS NOT NULL
    AND YEAR(o.OrderDate) = 2016
GROUP BY o.PickedByPersonID, p.FullName, p.IsEmployee
ORDER BY OrdersPicked DESC

(10 rows affected)

Total execution time: 00:00:00.011

PickedByPersonID,PickerName,PickerIsEmployee,OrdersPicked,FirstPick,LastPick
10,Stella Rosenhain,1,5,2016-01-05 11:00:00.0000000,2016-05-05 11:00:00.0000000
11,Ethan Onslow,1,5,2016-02-29 11:00:00.0000000,2016-05-20 11:00:00.0000000
4,Isabella Rupp,1,3,2016-02-22 11:00:00.0000000,2016-05-21 11:00:00.0000000
7,Amy Trefl,1,2,2016-01-27 11:00:00.0000000,2016-03-02 11:00:00.0000000
13,Hudson Hollinworth,1,2,2016-02-17 11:00:00.0000000,2016-03-15 11:00:00.0000000
9,Alica Fatnowna,1,2,2016-01-21 11:00:00.0000000,2016-01-28 11:00:00.0000000
3,Hudson Onslow,1,1,2016-02-26 11:00:00.0000000,2016-02-26 11:00:00.0000000
15,Taj Shand,1,1,2016-04-12 11:00:00.0000000,2016-04-12 11:00:00.0000000
8,Anthony Grosse,1,1,2016-05-13 11:00:00.0000000,2016-05-13 11:00:00.0000000
5,Eva Muirden,1,1,2016-02-08 11:00:00.0000000,2016-02-08 11:00:00.0000000
